In [1]:
import sqlite3
from datetime import datetime

In [2]:
# database 생성
# isolation_level : autocommit 여부
conn = sqlite3.connect("C:\\source\\pythonsource\\django\\kream\\db.sqlite3", isolation_level=None)

In [3]:
cursor = conn.cursor()

In [8]:
# 모든 데이터 삭제
cursor.execute('DELETE FROM kream_product')
cursor.execute('DELETE FROM kream_trade_total')

# 변경사항 저장
conn.commit()

In [10]:
cursor.execute('UPDATE sqlite_sequence SET seq = 0 WHERE name = "kream_product"')

In [11]:
from openpyxl import load_workbook
wb = load_workbook("./clean_data/soldout_clean_product2.xlsx")

# work sheet 가져오기
ws = wb.active

In [12]:
product_id_map = {}  # 제품 이름을 키로, 생성된 product_id를 값으로 저장
for row in ws.iter_rows(min_row=2, values_only=True):  # 첫 번째 행은 헤더라서 제외
    cursor.execute('''
        INSERT INTO kream_product (brand, name_kor, name_eng, model_no, release_date, color, img_url, product_detail_url, original_price)
        VALUES (?,?,?,?,?,?,?,?,?)
    ''',(row[0],row[1],row[2],row[3],row[4],row[5],row[7],row[8],row[6]))

conn.commit()

# Product 테이블에서 모든 제품 이름과 ID를 가져와서 매핑
cursor.execute('SELECT id, name_kor FROM kream_product')
product_id_map = {row[1]: row[0] for row in cursor.fetchall()}

In [4]:
# # 모든 데이터 삭제
# cursor.execute('DELETE FROM kream_product')

# # 변경사항 저장
# conn.commit()

# # 데이터베이스 연결 종료
# conn.close()

In [13]:
from openpyxl import load_workbook
wb = load_workbook("./clean_data/trade.xlsx")

# work sheet 가져오기
ws = wb.active

In [14]:
for row in ws.iter_rows(min_row=2, values_only=True):  # 첫 번째 행은 헤더라서 제외
    name_kor = row[0]  # 거래 데이터에서 제품 이름
    product_id = product_id_map.get(name_kor)
    cursor.execute('''
        INSERT INTO kream_trade_total (trade_size,trade_price,trade_year,trade_month,trade_hour,product_id)
        VALUES (?,?,?,?,?,?)
    ''', (row[2],row[3],row[6],row[7],row[8],product_id))

# 변경사항 저장
conn.commit()



In [9]:
# 변경할 행 업데이트
update_query = '''
UPDATE kream_trade_total
SET product_id = 214
WHERE product_id = 1560;
'''
cursor.execute(update_query)

# 변경 내용 커밋
conn.commit()

In [8]:
query = '''
SELECT * FROM kream_trade_total
WHERE id IS NULL;
'''
cursor.execute(query)